### IFN680 Sokoban Assignment


The functions and classes defined in this module will be called by a marker script. 
You should complete the functions and classes according to their specified interfaces.

You are not allowed to change the defined interfaces.
That is, changing the formal parameters of a function will break the 
interface and triggers to a fail for the test of your code.
 

In [ ]:
# Imports

import search
import sokoban
import tabooCells
from canGoThereSearch import can_go_there, CanGoThereSearch
from tabooCells import get_taboo_cells_positions
from collections import namedtuple
from SokobanPuzzle import SokobanPuzzle


In [ ]:
def my_team():
    '''
    Return the list of the team members of this assignment submission as a list
    of triplet of the form (student_number, first_name, last_name)
    
    '''
    return [(11771984, 'Daniel', 'Skauge'), (11771861, 'Erlend', 'Eiring')]

In [ ]:
def taboo_cells(warehouse):
    raise NotImplementedError()

In [ ]:
class SokobanPuzzle(search.Problem):
    raise NotImplementedError()

In [ ]:
def check_action_seq(warehouse, action_seq):
    raise NotImplementedError()

In [ ]:
def solve_sokoban_elem(warehouse):
    raise NotImplementedError()

In [ ]:
def can_go_there(warehouse, dst):
    raise NotImplementedError()

In [ ]:
def solve_sokoban_macro(warehouse):
    raise NotImplementedError()